<a href="https://colab.research.google.com/github/zaidlameer/DeetectorPrototype/blob/main/transformerPrototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"zaidthemler","key":"3b85272ca9bb36b4344d6af5456e904d"}'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images

Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
100% 1.68G/1.68G [00:21<00:00, 129MB/s]
100% 1.68G/1.68G [00:21<00:00, 84.5MB/s]


In [ ]:
import zipfile
import os

with zipfile.ZipFile("/content/deepfake-and-real-images.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset-folder")


In [ ]:
import random
import shutil
import os

def sample_dataset(input_dir, output_dir, sample_size_per_class):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for class_dir in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_dir)
        if os.path.isdir(class_path):
            # Randomly sample files
            sampled_files = random.sample(os.listdir(class_path), min(sample_size_per_class, len(os.listdir(class_path))))
            save_path = os.path.join(output_dir, class_dir)
            os.makedirs(save_path, exist_ok=True)
            for file in sampled_files:
                shutil.copy(os.path.join(class_path, file), os.path.join(save_path, file))

# Example usage
original_train_dir = "/content/dataset-folder/Dataset/Train"
# resized_train_dir = "/content/dataset-folder/ResizedDataset"
sampled_train_dir = "/content/dataset-folder/SampleDataset"
sample_dataset(original_train_dir, sampled_train_dir, sample_size_per_class=500)


# Paths

# resize_images(original_train_dir, resized_train_dir, size=(224, 224))

In [ ]:
from PIL import Image
import os

def resize_images(input_dir, output_dir, size=(224, 224)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)  # Create the main output directory if it doesn't exist
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('png', 'jpg', 'jpeg')):
                img = Image.open(os.path.join(root, file))
                img = img.resize(size, Image.ANTIALIAS)
                # Ensure the parent directory exists before saving
                save_path = os.path.join(output_dir, os.path.relpath(root, input_dir))
                os.makedirs(save_path, exist_ok=True)  # Create intermediate directories
                img.save(os.path.join(save_path, file))

# Example usage
sampled_train_dir = "/content/dataset-folder/SampledDataset"
resized_train_dir = "/content/dataset-folder/ResizedDataset"
resize_images(sampled_train_dir, resized_train_dir, size=(224, 224))


In [ ]:
# we gonna use the sample dataset for this use case
from torchvision import transforms, datasets
import torch

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Dataset paths
train_dir = "/content/dataset-folder/Dataset/Test" # using the testing dataset to train
test_dir = "/content/dataset-folder/SampleDataset"

# Load datasets
train_data = datasets.ImageFolder(root=train_dir, transform=transform)
test_data = datasets.ImageFolder(root=test_dir, transform=transform)

# Data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
!pip install torch torchvision linformer
!pip install torch torchvision transformers



In [ ]:
!pip install tensorflow numpy pandas


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dataset_dir = '/content/dataset-folder/datasetDir'
train_dir = '/content/dataset-folder/datasetDir/train'
val_dir = '/content/dataset-folder/datasetDir/train'
test_dir = '/content/dataset-folder/datasetDir/test'

# Create directories
for directory in [train_dir, val_dir, test_dir]:
    os.makedirs(directory, exist_ok=True)

# Get image paths
images = [os.path.join(dataset_dir, img) for img in os.listdir(dataset_dir)]

# Split the dataset
train_images, temp_images = train_test_split(images, test_size=0.3)
val_images, test_images = train_test_split(temp_images, test_size=0.5)

# Move images to respective directories
for img in train_images:
    shutil.move(img, os.path.join(train_dir, os.path.basename(img)))
for img in test_images:
    shutil.move(img, os.path.join(test_dir, os.path.basename(img)))


ValueError: With n_samples=1, test_size=0.5 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.